In [19]:
import random
import numpy as np
from numpy.random import rand

class Patient:
    def __init__(self, arrival_time, disability_type, destination):
        self.arrival_time = arrival_time
        self.disability_type = disability_type
        self.destination = destination
    def __str__(self) -> str:
        return "arrival : " + str(self.arrival_time)

class Server:
    def __init__(self, is_robotic=False, alpha=1.0, error_prob=0.0):
        self.is_robotic = is_robotic
        self.alpha = alpha
        self.error_prob = error_prob
        self.busy_until = 0

    def serve(self, patient, current_time):
        # Determine service time based on disability type and destination
        service_times = {
            1: {'A': 8, 'B': 10},
            2: {'A': 10, 'B': 14},
            3: {'A': 18, 'B': 16},
            'None': {'A': 4, 'B': 6}
        }
        disability = patient.disability_type if patient.disability_type else 'None'
        base_time = service_times[disability][patient.destination]
        service_time = base_time * (self.alpha if self.is_robotic and disability != 'None' else 1)
        # Add error-related delays
        errors = np.random.geometric(1-self.error_prob) - 1 if self.error_prob > 0 else 0
        total_service_time = service_time * (1 + errors)
        self.busy_until = current_time + total_service_time
        return total_service_time

class Simulation:
    def __init__(self, total_time, arrival_rate, disability_prob, destination_prob, robot_alpha, error_prob, is_robotic):
        self.total_time = total_time
        self.arrival_rate = arrival_rate
        self.disability_prob = disability_prob
        self.destination_prob = destination_prob
        self.robot_alpha = robot_alpha
        self.error_prob = error_prob
        self.is_robotic = is_robotic
        self.patients = []
        self.standard_server = Server(is_robotic=False, error_prob=error_prob)
        self.robotic_server = Server(is_robotic=is_robotic, alpha=robot_alpha, error_prob=error_prob)

    def generate_patients(self):
        current_time = 0
        while current_time < self.total_time:
            # Generate next arrival time
            inter_arrival_time = np.random.exponential(1 / self.arrival_rate)
            current_time += inter_arrival_time
            if current_time >= self.total_time:
                break

            # Assign disability type and destination
            disability = random.choices([1, 2, 3, 'None'], weights=[self.disability_prob/3, self.disability_prob/3, self.disability_prob/3, 1 - self.disability_prob])[0]

            destination = random.choices(['A', 'B'], weights=[self.destination_prob, 1 - self.destination_prob])[0]

            self.patients.append(Patient(current_time, disability, destination))

    def select_pationt_q2(self, queue2):
        if not self.is_robotic:
            return queue2.pop(0)

        for i,patient in enumerate(queue2):
            if patient.disability_type != 'None':
                return queue2.pop(i)

        return queue2.pop(0)



    def run(self):
        queue1 = []
        queue2 = []
        current_time = 0
        waiting_times = []
        waiting_times_dis = []
        waiting_times_not_dis = []
        service_times = []
        utilize_1 = 0
        utilize_2 = 0



        for patient in self.patients:
            # Fast-forward time
            current_time = max(current_time, patient.arrival_time)
            if patient.disability_type == 'None':
            # Add patient to the queue
                if len(queue2) == len(queue1):
                    rand_q = random.choices([1, 2])[0]
                    if rand_q == 1:
                        queue1.append(patient)
                    else:
                        queue2.append(patient)


                elif len(queue2) > len(queue1):
                    queue1.append(patient)
                else:
                    queue2.append(patient)

            else:
                queue2.append(patient)

            # Serve patients
            while (queue1 and current_time >= self.standard_server.busy_until):
                patient_to_serve = queue1.pop(0)
                server = self.standard_server

                waiting_times.append(max(server.busy_until,patient_to_serve.arrival_time) - patient_to_serve.arrival_time)
                if patient_to_serve.disability_type == 'None':
                    waiting_times_not_dis.append(max(server.busy_until,patient_to_serve.arrival_time) - patient_to_serve.arrival_time)
                else:
                    waiting_times_dis.append(max(server.busy_until,patient_to_serve.arrival_time) - patient_to_serve.arrival_time)

                service_time = server.serve(patient_to_serve, max(server.busy_until,patient_to_serve.arrival_time))
                utilize_1 += service_time


                service_times.append(service_time)
                current_time += service_time

            while  (queue2 and current_time >= self.robotic_server.busy_until):

                patient_to_serve = self.select_pationt_q2(queue2)
                server = self.robotic_server

                waiting_times.append(max(server.busy_until,patient_to_serve.arrival_time) - patient_to_serve.arrival_time)
                if patient_to_serve.disability_type == 'None':
                    waiting_times_not_dis.append(max(server.busy_until,patient_to_serve.arrival_time) - patient_to_serve.arrival_time)
                else:
                    waiting_times_dis.append(max(server.busy_until,patient_to_serve.arrival_time) - patient_to_serve.arrival_time)

                service_time = server.serve(patient_to_serve, max(server.busy_until,patient_to_serve.arrival_time))

                utilize_2 += service_time

                service_times.append(service_time)
                current_time += service_time


        # Calculate metrics
        metrics = {
            'avg_waiting_time': np.mean(waiting_times),
            'avg_service_time': np.mean(service_times),
            'avg_queue_length': np.sum(waiting_times)/(2*current_time),
            'avg_total_time_in_system': np.mean(waiting_times) + np.mean(service_times),
            'avg_waiting_time_disabled': np.mean(waiting_times_dis),
            'avg_waiting_time_not_disabled': np.mean(waiting_times_not_dis),
            'number_of_disabled': len(waiting_times_dis),
            'number_of_not_disabled': len(waiting_times_not_dis),
            'utilization1': utilize_1/current_time,
            'utilization2': utilize_2/current_time,

        }

        return metrics

def run_simulation_once(args):
    simulation = Simulation(
        **args
    )
    simulation.generate_patients()
    metrics = simulation.run()
    return metrics


def run_multiple_times(times, args):
    results = []
    for i in range(times):
        results.append(run_simulation_once(args))

    ans = {k:np.mean([val[k] for val in results]) for k in results[0].keys()}

    print('average metrics are')
    for k in ans:
        print('   ',k,ans[k])

    print('----------------------')




sample run for testing:

In [2]:
number_of_runs = 300
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.2,  # Lambda for inter-arrival times
    'disability_prob':0.3,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.05,  # Probability of error
    'is_robotic':True,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 7.618815231381671
    avg_service_time 5.834980893283146
    avg_queue_length 0.6411869944382762
    avg_total_time_in_system 13.453796124664818
    avg_waiting_time_disabled 10.766302653213057
    avg_waiting_time_not_disabled 6.236200000768734
    number_of_disabled 59.61
    number_of_not_disabled 139.46
    utilization1 0.3241720130218415
    utilization2 0.6595620934522783
----------------------


we run the sumulation for both is-roboic true and false with different parameters and compare results:

In [3]:
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.2,  # Lambda for inter-arrival times
    'disability_prob':0.3,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.05,  # Probability of error
    'is_robotic':True,
}
run_multiple_times(number_of_runs, args)
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.2,  # Lambda for inter-arrival times
    'disability_prob':0.3,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.05,  # Probability of error
    'is_robotic':False,
}
run_multiple_times(number_of_runs, args)




average metrics are
    avg_waiting_time 8.533890882165933
    avg_service_time 5.851223905380036
    avg_queue_length 0.7163298247546979
    avg_total_time_in_system 14.38511478754597
    avg_waiting_time_disabled 12.016222647438848
    avg_waiting_time_not_disabled 7.000455593440114
    number_of_disabled 60.916666666666664
    number_of_not_disabled 140.61666666666667
    utilization1 0.318961053982723
    utilization2 0.6644078125855385
----------------------
average metrics are
    avg_waiting_time 78.21296476921111
    avg_service_time 7.865614859858092
    avg_queue_length 4.883095452929618
    avg_total_time_in_system 86.07857962906921
    avg_waiting_time_disabled 115.75987970969982
    avg_waiting_time_not_disabled 61.444504109694435
    number_of_disabled 59.85333333333333
    number_of_not_disabled 139.53333333333333
    utilization1 0.24165368163520767
    utilization2 0.7516793541038984
----------------------


In [4]:
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.1,  # Lambda for inter-arrival times
    'disability_prob':0.5,  # Probability of disability
    'destination_prob':0.7,  # Probability of destination A
    'robot_alpha':0.3,  # Alpha for robotic server
    'error_prob':0.5,  # Probability of error
    'is_robotic':True,
}
run_multiple_times(number_of_runs, args)
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.1,  # Lambda for inter-arrival times
    'disability_prob':0.5,  # Probability of disability
    'destination_prob':0.7,  # Probability of destination A
    'robot_alpha':0.3,  # Alpha for robotic server
    'error_prob':0.5,  # Probability of error
    'is_robotic':False,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 7.856609552970852
    avg_service_time 8.407391924204626
    avg_queue_length 0.38526969120450727
    avg_total_time_in_system 16.264001477175476
    avg_waiting_time_disabled 9.806921498037598
    avg_waiting_time_not_disabled 5.902719181009905
    number_of_disabled 49.45333333333333
    number_of_not_disabled 50.42333333333333
    utilization1 0.22707005037895375
    utilization2 0.5862933431761718
----------------------
average metrics are
    avg_waiting_time 191.27909882288807
    avg_service_time 16.954077859577833
    avg_queue_length 5.437432385609629
    avg_total_time_in_system 208.23317668246588
    avg_waiting_time_disabled 253.08963719048216
    avg_waiting_time_not_disabled 127.43961642133131
    number_of_disabled 49.55
    number_of_not_disabled 50.17
    utilization1 0.13757949770938446
    utilization2 0.8493256047082884
----------------------


as you see in the results in all combinations using robots helps us in significantlly reducing the time needed for our customers

lets play with parameters:

In [5]:
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.5,  # Lambda for inter-arrival times
    'disability_prob':0.3,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.05,  # Probability of error
    'is_robotic':True,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 328.01031945476694
    avg_service_time 5.851334513547429
    avg_queue_length 27.994778806320888
    avg_total_time_in_system 333.8616539683144
    avg_waiting_time_disabled 486.8275889296172
    avg_waiting_time_not_disabled 259.62556736182785
    number_of_disabled 150.23333333333332
    number_of_not_disabled 350.9633333333333
    utilization1 0.3278967365256393
    utilization2 0.6713986271229243
----------------------


In [6]:
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.1,  # Lambda for inter-arrival times
    'disability_prob':0.3,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.05,  # Probability of error
    'is_robotic':True,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 1.6639225687659216
    avg_service_time 5.848620700472036
    avg_queue_length 0.08484185228834763
    avg_total_time_in_system 7.512543269237956
    avg_waiting_time_disabled 2.197101066684464
    avg_waiting_time_not_disabled 1.4300622742757303
    number_of_disabled 29.983333333333334
    number_of_not_disabled 69.99
    utilization1 0.19011168857715183
    utilization2 0.39392372697584294
----------------------


In [7]:
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.5,  # Lambda for inter-arrival times
    'disability_prob':0.3,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.05,  # Probability of error
    'is_robotic':False,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 659.1852387007992
    avg_service_time 7.875932964006286
    avg_queue_length 41.7609486431573
    avg_total_time_in_system 667.0611716648057
    avg_waiting_time_disabled 991.551257912731
    avg_waiting_time_not_disabled 515.7054905834589
    number_of_disabled 150.43
    number_of_not_disabled 351.1666666666667
    utilization1 0.24280468280304987
    utilization2 0.7566816175570676
----------------------


In [8]:
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.1,  # Lambda for inter-arrival times
    'disability_prob':0.3,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.8,  # Probability of error
    'is_robotic':False,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 632.1597718143014
    avg_service_time 37.38333465179041
    avg_queue_length 8.315750148340932
    avg_total_time_in_system 669.5431064660918
    avg_waiting_time_disabled 905.0054501819247
    avg_waiting_time_not_disabled 510.4957035720683
    number_of_disabled 30.006666666666668
    number_of_not_disabled 69.61666666666666
    utilization1 0.24144639343603724
    utilization2 0.7547990423698064
----------------------


In [9]:
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.1,  # Lambda for inter-arrival times
    'disability_prob':0.1,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.9,  # Alpha for robotic server
    'error_prob':0.05,  # Probability of error
    'is_robotic':True,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 1.7296994863922275
    avg_service_time 6.140259106044611
    avg_queue_length 0.08750875842983134
    avg_total_time_in_system 7.869958592436839
    avg_waiting_time_disabled 2.413869747432585
    avg_waiting_time_not_disabled 1.6407063487220501
    number_of_disabled 10.116666666666667
    number_of_not_disabled 90.22
    utilization1 0.24673984821909792
    utilization2 0.36837410265859
----------------------


In [10]:
args = {
    'total_time':10000,  # Total simulation time
    'arrival_rate':0.1,  # Lambda for inter-arrival times
    'disability_prob':0.6,  # Probability of disability
    'destination_prob':0.6,  # Probability of destination A
    'robot_alpha':0.6,  # Alpha for robotic server
    'error_prob':0.1,  # Probability of error
    'is_robotic':True,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 5.767425081570862
    avg_service_time 7.1393997640159235
    avg_queue_length 0.28878523288570423
    avg_total_time_in_system 12.906824845586785
    avg_waiting_time_disabled 7.120226908221701
    avg_waiting_time_not_disabled 3.740463424379153
    number_of_disabled 598.9966666666667
    number_of_not_disabled 400.66
    utilization1 0.10666145921152208
    utilization2 0.6064431901087795
----------------------


now lets see what is effect of error prob:

In [11]:
number_of_runs = 300
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.2,  # Lambda for inter-arrival times
    'disability_prob':0.3,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.00,  # Probability of error
    'is_robotic':True,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 5.890633610818303
    avg_service_time 5.5539972098572346
    avg_queue_length 0.5172102600376913
    avg_total_time_in_system 11.444630820675535
    avg_waiting_time_disabled 8.22287624252524
    avg_waiting_time_not_disabled 4.8896743877456315
    number_of_disabled 58.513333333333335
    number_of_not_disabled 139.62
    utilization1 0.3217442802090665
    utilization2 0.6527858848107919
----------------------


In [12]:
number_of_runs = 300
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.2,  # Lambda for inter-arrival times
    'disability_prob':0.3,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.05,  # Probability of error
    'is_robotic':True,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 8.146271480270967
    avg_service_time 5.8441057093434905
    avg_queue_length 0.6840472426768671
    avg_total_time_in_system 13.990377189614458
    avg_waiting_time_disabled 11.434360142292228
    avg_waiting_time_not_disabled 6.70181739088462
    number_of_disabled 59.67
    number_of_not_disabled 139.35
    utilization1 0.3221913371428876
    utilization2 0.6618085935220884
----------------------


In [13]:
number_of_runs = 300
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.2,  # Lambda for inter-arrival times
    'disability_prob':0.3,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.1,  # Probability of error
    'is_robotic':True,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 10.533585523134304
    avg_service_time 6.182137240682238
    avg_queue_length 0.8379069242313821
    avg_total_time_in_system 16.715722763816544
    avg_waiting_time_disabled 14.855694319196006
    avg_waiting_time_not_disabled 8.640971734020441
    number_of_disabled 59.74
    number_of_not_disabled 139.93666666666667
    utilization1 0.325585242411918
    utilization2 0.6608718622688563
----------------------


In [14]:
number_of_runs = 300
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.2,  # Lambda for inter-arrival times
    'disability_prob':0.3,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.2,  # Probability of error
    'is_robotic':True,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 22.201541113201213
    avg_service_time 6.920446722993963
    avg_queue_length 1.577715149531863
    avg_total_time_in_system 29.121987836195174
    avg_waiting_time_disabled 31.91072628333263
    avg_waiting_time_not_disabled 17.969787359942238
    number_of_disabled 59.016666666666666
    number_of_not_disabled 140.31
    utilization1 0.328777736201043
    utilization2 0.6624741750504215
----------------------


In [15]:
number_of_runs = 300
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.2,  # Lambda for inter-arrival times
    'disability_prob':0.3,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.5,  # Probability of error
    'is_robotic':True,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 180.1404202876817
    avg_service_time 11.08435628681913
    avg_queue_length 8.059142791838466
    avg_total_time_in_system 191.22477657450085
    avg_waiting_time_disabled 264.4544570216654
    avg_waiting_time_not_disabled 143.09185881905597
    number_of_disabled 60.656666666666666
    number_of_not_disabled 140.88666666666666
    utilization1 0.324841309248225
    utilization2 0.672007182188273
----------------------


In [16]:
number_of_runs = 300
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.2,  # Lambda for inter-arrival times
    'disability_prob':0.3,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.8,  # Probability of error
    'is_robotic':True,
}
run_multiple_times(number_of_runs, args)

average metrics are
    avg_waiting_time 1040.0268041109825
    avg_service_time 27.59824853207802
    avg_queue_length 18.770721490560863
    avg_total_time_in_system 1067.6250526430606
    avg_waiting_time_disabled 1370.3464943470049
    avg_waiting_time_not_disabled 895.2561060288762
    number_of_disabled 60.45
    number_of_not_disabled 139.89666666666668
    utilization1 0.3253326819471325
    utilization2 0.6736256568549485
----------------------


we see that avg time of service is times 1/(1-error prob). that is what we expect to see theorically.
but waiting time increases dramatically and effect is almost exponential.

now we want to comapre effect of prioritizing disable people:

In [17]:
class SimulationNoPr(Simulation):
    def select_pationt_q2(self, queue2):

        return queue2.pop(0)


In [20]:
args = {
    'total_time':1000,  # Total simulation time
    'arrival_rate':0.2,  # Lambda for inter-arrival times
    'disability_prob':0.4,  # Probability of disability
    'destination_prob':0.4,  # Probability of destination A
    'robot_alpha':0.5,  # Alpha for robotic server
    'error_prob':0.05,  # Probability of error
    'is_robotic':True,
}
print('with prioritizing disables')
run_multiple_times(number_of_runs, args)


def run_simulation_once(args):
    simulation = SimulationNoPr(
        **args
    )
    simulation.generate_patients()
    metrics = simulation.run()
    return metrics


def run_multiple_times(times, args):
    results = []
    for i in range(times):
        results.append(run_simulation_once(args))

    ans = {k:np.mean([val[k] for val in results]) for k in results[0].keys()}

    print('average metrics are')
    for k in ans:
        print('   ',k,ans[k])

    print('----------------------')
run_multiple_times(number_of_runs, args)



with prioritizing disables
average metrics are
    avg_waiting_time 13.782641400822849
    avg_service_time 5.980306865309527
    avg_queue_length 1.1322267465090836
    avg_total_time_in_system 19.762948266132373
    avg_waiting_time_disabled 18.68849266339084
    avg_waiting_time_not_disabled 10.381804577452629
    number_of_disabled 80.75
    number_of_not_disabled 119.33666666666667
    utilization1 0.26770297279291677
    utilization2 0.7173579630951936
----------------------
average metrics are
    avg_waiting_time 14.220261966760367
    avg_service_time 5.992492171166009
    avg_queue_length 1.1670571488833028
    avg_total_time_in_system 20.212754137926375
    avg_waiting_time_disabled 19.333574605328007
    avg_waiting_time_not_disabled 10.762202418984806
    number_of_disabled 80.04
    number_of_not_disabled 121.02
    utilization1 0.26971311607537546
    utilization2 0.7159069226735995
----------------------


we see that with prioritizing disables the queue is a little bit less heavy and we have less waiting times